In [1]:
import os 
os.environ["SPARK_HOME"] = ""
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.2.10 pyspark-shell'

import findspark

# findspark.add_packages(
#     ['org.postgresql:postgresql:42.2.10']
# )
findspark.add_jars(
    [
        'postgresql-42.6.0.jar'
    ]
)
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.functions import col, avg

spark = SparkSession.builder\
    .appName("capstone_project")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/05/24 13:28:29 WARN Utils: Your hostname, pcminh-ThinkPad resolves to a loopback address: 127.0.1.1; using 172.31.163.12 instead (on interface eth0)
23/05/24 13:28:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/24 13:28:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/24 13:28:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
from batch.rw import read_table_to_view
read_table_to_view(spark)
spark.sql("SHOW TABLES").show()


23/05/24 13:28:40 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |     crashes|       true|
|         |redlight_cam|       true|
|         |   speed_cam|       true|
|         |     streets|       true|
|         |traffic_hist|       true|
+---------+------------+-----------+



In [5]:
from batch.dimensions import dim_street, dim_camera_stage, dim_camera_map_street, dim_segment, dim_date, dim_hour
from batch.facts import fact_violation, fact_crash, fact_congestion_batch

In [9]:
d_street = dim_street(spark)

In [7]:
d_camera = dim_camera_stage(spark)
d_camera = dim_camera_map_street(spark, d_camera, d_street)

In [10]:
d_segment = dim_segment(spark, d_street)

In [12]:
d_segment.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_segment.csv')

23/05/24 13:31:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:31:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:32:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Full Street Name, Direction, Street , Suffix
 Schema: full_street_name, direction, street, suffix
Expected: full_street_name but found: Full Street Name
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/streets/streets.csv
23/05/24 13:32:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/24 13:32:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [12]:
d_date = dim_date(spark)

In [15]:
d_hour = dim_hour(spark)

In [6]:
f_violation = fact_violation(spark)

In [13]:
f_violation.describe()

DataFrame[summary: string, camera_id: string, violation_date_key: string, violation_type: string, violations: string]

In [7]:
f_violation.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/f_violation.csv')

23/05/24 13:28:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CAMERA ID, VIOLATION DATE, VIOLATIONS
 Schema: camera_id, violation_date_mmddyyyy, violations
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/redlight_cam/redlight_cam.csv
23/05/24 13:28:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: CAMERA ID, VIOLATION DATE, VIOLATIONS
 Schema: camera_id, violation_date_mmddyyyy, violations
Expected: camera_id but found: CAMERA ID
CSV file: file:///home/pcminh/bigdata-capstone/data/raw/traffic/speed_cam/speed_cam.csv


In [19]:
d_date.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_date.csv')
d_hour.repartition(1).write.format('csv').mode('overwrite').save('./data/csv/d_hour.csv')